In [ ]:
from model.Attention_model import build_transformer_model
import time
import torch
import os
from sklearn.metrics import accuracy_score
from util import load_dataset
import numpy as np
import wandb
import yaml
# Authenticate with your API key
wandb.login(key="4dd27c7624f2ab82554553d3e872b47dcaa05780")


def run_epoch(model, optimizer, data_loader, loss_func, device, results, score_funcs, prefix="", desc=None):   
    
    model = model.to(device)
    running_loss = []
    y_true = []
    y_pred = []
    start = time.time()
    for inputs, labels in (data_loader):
        # -- Move the batch to the device we are using.
        inputs = moveTo(inputs, device)
        labels = moveTo(labels, device)

        if prefix == "validation" or prefix == "test":
            inputs.requires_grad_(False)  # Ensure inputs don't track gradients
            labels.requires_grad_(False)  # Ensure labels don't track gradients

        # -- Output of the model
        y_hat = model(inputs)

        # -- Compute loss.
        loss = loss_func(y_hat, labels)

        # -- Training?
        if model.training:
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        # -- Now we are just grabbing some information we would like to have
        running_loss.append(loss.item())

        if len(score_funcs) > 0 and isinstance(labels, torch.Tensor):
            # -- moving labels & predictions back to CPU for computing / storing predictions
            labels = labels.detach().cpu().numpy()
            y_hat = y_hat.detach().cpu().numpy()
            # -- add to predictions so far
            y_true.extend(labels.tolist())
            y_pred.extend(y_hat.tolist())
    # -- end of one training epoch
    end = time.time()

    y_pred = np.asarray(y_pred)
    # We have a classification problem, convert to labels
    if len(y_pred.shape) == 2 and y_pred.shape[1] > 1:
        y_pred = np.argmax(y_pred, axis=1)
    # Else, we assume we are working on a regression problem

    results[prefix + " loss"].append(np.mean(running_loss))
    for name, score_func in score_funcs.items():
        try:
            results[prefix + " " + name].append(score_func(y_true, y_pred))
        except:
            results[prefix + " " + name].append(float("NaN"))
    return end-start  # time spent on epoch


def train_model(epoches,
                model,
                optimizer,
                train_loader,
                loss_func,
                score_funcs,
                result_path,
                patch_size,
                validation_loader=None,
                test_loader=None):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # -- Create Result file
    if os.path.exists(result_path) is not True:
        os.mkdir(result_path)

    # -- save all results
    checkpoint_file_results = os.path.join(
        result_path, ('All_results_'+str(patch_size) + '_patchsize.pt'))
    # -- save the best result based on validation accuracy
    checkpoint_file_best_result = os.path.join(
        result_path, ('BestResult_' + str(patch_size) + '_patchsize.pt'))

    # -- send model on the device
    model = model.to(device)
    to_track = ["epoch", "total time", "train Accuracy", "train loss"]

    # -- There is Validation loader?
    if validation_loader is not None:
        to_track.append("validation Accuracy")
        to_track.append("validation loss")

    # -- There is test loader ?
    if test_loader is not None:
        to_track.append("test Accuracy")
        to_track.append("test loss")

    total_train_time = 0
    results = {}

    # -- Initialize every item with an empty list
    for item in to_track:
        results[item] = []

    Best_validation_Accuracy = 0.0

    # -- Train model
    print('Training begins...\n')

    for epoch in range(epoches):
        # -- set the model on train
        model = model.train()
        # -- Train for one epoch
        total_train_time += run_epoch(model, optimizer, train_loader,
                                      loss_func, device, results,
                                      score_funcs, prefix="train", desc="Training")

        # -- Save epoch and processing time
        results["epoch"].append(epoch)
        results["total time"].append(total_train_time)

        #   ******  Validating  ******
        if validation_loader is not None:
            model = model.eval()  # Set the model to "evaluation" mode
            with torch.no_grad():
                run_epoch(model, optimizer, validation_loader,
                          loss_func, device, results,
                          score_funcs, prefix="validation", desc="Validating")

        #   ******  Testing  ******
        if test_loader is not None:
            model = model.eval()
            with torch.no_grad():
                run_epoch(model, optimizer, test_loader,
                          loss_func, device, results,
                          score_funcs, prefix="test", desc="Testing")

        #   ******  Save results of each epoch  ******
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'results': results
        }, checkpoint_file_results)
        # show the progress and metrics
        print('\nEpoch: {}   Training accuracy: {:.2f}   Validation accuracy: {:.2f}   Test Accuracy: {:.2f}'
              .format(epoch, results['train Accuracy'][-1]*100, results['validation Accuracy'][-1]*100, results['test Accuracy'][-1]*100))
        # save the model based on the validation accuracy
        if results['validation Accuracy'][-1] > Best_validation_Accuracy:
            print('\nEpoch: {}   Training accuracy: {:.2f}   best Val accuracy: {:.2f}   Test Accuracy: {:.2f}'
                  .format(epoch, results['train Accuracy'][-1]*100, results['validation Accuracy'][-1]*100, results['test Accuracy'][-1]*100))
            Best_validation_Accuracy = results['validation Accuracy'][-1]
            best_result = {}
            best_result["epoch"] = []
            best_result["train accuracy"] = []
            best_result["validation accuracy"] = []
            best_result["test accuracy"] = []

            best_result["epoch"].append(epoch)
            best_result["train accuracy"].append(results['train Accuracy'][-1])
            best_result["validation accuracy"].append(
                results['validation Accuracy'][-1])
            best_result["test accuracy"].append(results['test Accuracy'][-1])

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'results': best_result
            }, checkpoint_file_best_result)

In [ ]:
from util import get_dataset

current_location = os.getcwd()
with open (current_location+'/config/config.yml', 'r') as file:
        config = yaml.safe_load(file)

# Initialize W&B
wandb.init(
    project="translation",  # Name of your project
    config={
        "learning_rate": config['TRAIN']['lr'],
        "batch_size": config['TRAIN']['batch_size'],
        "epochs": config['TRAIN']['epochs'],
        "model": 'english_to_french',
    })

Result_Directory = os.path.join(config['BENCHMARK']['results_path'], config['BENCHMARK']['model_name'])  
os.makedirs(Result_Directory, exist_ok=True)

score_funcs = {"Accuracy": accuracy_score}
# getting the dataloaders
train_dataloader, val_dataloader, test_dataloader, source_tokenizer, target_tokenizer =  get_dataset(config)


In [4]:
# get model
from model.Attention_model import build_transformer_model
model = build_transformer_model(config)
# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = config['TRAIN']['lr'], eps = 1e-9)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('opus_books', 'en-fr', split='train')

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
dataset[0]['translation']['en']

In [ ]:
dataset[0]['translation']['fr']

In [1]:
import torch

In [2]:
from torch.utils.data import DataLoader # type: ignore
file_path='./dataloaders/test_dataloader.pth'
checkpoint = torch.load(file_path)
# Extract the test_dataset
test_dataset = checkpoint['test_dataset']
# Recreate the test DataLoader with batch_size = 1 and shuffle = False
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

/tmp/ipykernel_347033/3730158998.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(file_path)
/opt/conda/v3/envs/transformer_env/lib/python3.8/site

In [3]:
len(test_dataloader)

12709

In [5]:
import numpy as np
random_samples_idx =np.random.randint(0,len(test_dataloader),3)
random_samples_idx

array([6384,  534, 7277])

In [8]:
import os
os.getcwd()

'/media/saeid/Crucial/ML_Projects/GitHub_Projects/Attention_From_Scratch'